In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import modules.image_importer as ii
import modules.prepare_images as reform


In [ ]:
paths_images_training = ii.image_paths("fish_images/copy_fish_images")

In [ ]:
##TODO - vurdér om du skal have dette med Cathrine!!!!!!!!!!!!!!!!!!!!!!

img_loc = './fish_images/Aborre 1.png'
# 1
img = cv.imread(img_loc, cv.IMREAD_UNCHANGED)
print(img.shape)
# 2
img_grey = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
print(img_grey.shape)
# 3
thresh = 90 # the lower the threshhold the more details we get
ret,thresh_img = cv.threshold(img_grey, thresh, 255, cv.THRESH_BINARY)
# 4
contours, hierarchy = cv.findContours(thresh_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
#create an empty image for contours
img_contours = np.zeros(img.shape)
# draw the contours on the empty image
cv.drawContours(img_contours, contours, -1, (255,255,255), 3)
#save image
cv.imwrite('./object_detection/input/test_conture.png',img_contours) 
plt.imshow(img_contours, interpolation='none')

In [ ]:
import requests
import bs4
import url_fish as fish

"""The xml-file url is not allowed to go on public online"""
xml_file_url = fish.get_fish_xml()

def get_spicies(xml_file):
    r = requests.get(xml_file)
    if r.status_code == 200:
        soup = bs4.BeautifulSoup(r.content, 'xml')
        spiecies_tag = soup.findAll('navn')
        return[spiecies.text for spiecies in spiecies_tag]
    
spiecies = get_spicies(xml_file_url)

In [ ]:
import os

img_size = 50
def get_data(images_dir):
    images = [] 
    for label in spiecies: 
        for img in os.listdir(images_dir):
            if (label in img):
                class_num = spiecies.index(label)
                try:
                    img_arr = cv.imread(os.path.join(images_dir, img))
                    resized_image = reform.resizing_image_squared(os.path.join(images_dir, img), img_size)
                    images.append([resized_image, class_num])
                except Exception as e:
                    print(e)
    return np.array(images)

training_imgs = get_data("./fish_images/copy_fish_images")
print("ok")

In [ ]:
##Kladde bliver ikke
import seaborn as sns

print(len(training_imgs))

lst = []
for i in training_imgs:
    if(i[1] == 0):
        lst.append(spiecies[0])
    elif (i[2] == 0):
        lst.append(spiecies[1])
    elif (i[3] == 0):
        lst.append(spiecies[2])
    elif (i[4] == 0):
        lst.append(spiecies[3])
    elif (i[5] == 0):
        lst.append(spiecies[4])
    elif (i[6] == 0):
        lst.append(spiecies[5])
    elif (i[7] == 0):
        lst.append(spiecies[6])
    elif (i[8] == 0):
        lst.append(spiecies[7])
    elif (i[9] == 0):
        lst.append(spiecies[8])
    elif (i[10] == 0):
        lst.append(spiecies[9])
    else:
        lst.append("No fish")
sns.set_style('darkgrid')
sns.countplot(l)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

In [ ]:
print(len(paths_images_training))
for image_path in paths_images_training:
    resized_image = reform.resizing_image_squared(image_path, 50)
    image_title = image_path.replace("fish_images/copy_fish_images/","")
    image_name = image_title.replace(".png","")
    print(image_name)
    plt.imshow(resized_image, cmap=plt.cm.binary)
    #ii.saving_image("fish_images/training_images",resized_image,image_name)